In [3]:
import re
import networkx as nx
import matplotlib.colors as clr
from utils.graph import Graph_Viz_Engine
from utils.io import read_in


In [66]:
article_relations = './data/outbound-requests-by-domain-2.json'
relations = read_in(article_relations)

In [67]:
def extract_domain(url: str) -> str:
    
    pattern = r"(((([A-Za-z0-9]+){1,63}\.)|(([A-Za-z0-9]+(\-)+[A-Za-z0-9]+){1,63}\.))+){1,255}"
    domain = re.search(pattern, url)

    return domain.group().replace('www', '').replace('.', '')

In [68]:
n_relations = {}

for idx, key in enumerate(relations.keys()):
    n_key= extract_domain(key)

    n_relations[n_key] = relations[key]

In [77]:
t_blob = n_relations['nytimes']['content']

In [ ]:
# analyze these text together and make a graph

In [78]:
from textblob import TextBlob

In [79]:
nyt_blob = TextBlob(t_blob)

In [86]:
polarities = []

for sentence in nyt_blob.sentences:
    polarities.append(sentence.sentiment.polarity)